# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

# Import Dependencies

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from api_keys import g_key

# Configure Google Maps

In [2]:
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv("weatherData.csv")
df.head()

,City,Latitude,Longitude,Temperature,Humidity,Pressure,Wind Speed,Cloudiness
0,port alfred,-33.59,26.89,15.56,89,1015,3.30,0
1,atuona,-9.80,-139.03,27.80,79,1011,7.82,2
2,vaini,-21.20,-175.20,27.00,74,1013,5.10,20
3,punta arenas,-53.15,-70.92,7.89,87,1011,6.70,20
4,rikitea,-23.12,-134.97,26.06,84,1013,6.75,100


# Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = df[["Latitude", "Longitude"]]

humid = df["Humidity"]

In [5]:
figure_layout = {
    'width': '950px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humid)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

# Ideal Weather Conditions

### Create new DataFrame matching Ideal Weather Conditions

DataFrame will contain cities with:
- Temperatures between 21℃ and 26℃
- Wind Speed less than 10
- Cloudiness 10%

In [6]:
mask1 = (df.Temperature >= 21) & (df.Temperature < 26)
mask2 = df["Wind Speed"] < 10
mask3 = df.Cloudiness == 0
mask4 = mask1 & mask2 & mask3

ideal_cities = df.loc[mask4].reset_index(drop=True)
ideal_cities.head()

,City,Latitude,Longitude,Temperature,Humidity,Pressure,Wind Speed,Cloudiness
0,soto la marina,23.77,-98.22,25.00,80,1014,5.36,0
1,vila velha,-20.33,-40.29,21.18,94,1018,1.50,0
2,redlands,34.06,-117.18,22.24,52,1008,1.79,0
3,katsuura,35.13,140.30,24.27,60,1013,6.70,0
4,tiznit,29.58,-9.50,21.07,44,1017,0.70,0


### Number of Ideal Cities

In [7]:
len(ideal_cities)

10

### Map of Ideal Cities 1

- markers to illustrate locations of ideal cities

In [8]:
figure_layout = {
    'width': '950px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(map_type="TERRAIN", layout=figure_layout)

coordinates = ideal_cities[["Latitude", "Longitude"]]

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Map of Ideal Cities 2

- include pop-up with details on ideal cities

In [9]:
city_info = []

for indx, row in ideal_cities.iterrows():
    info_box = f"""
                <dl>
                <dt>Name</dt><dd>{row.City}</dd>
                <dt>Temperature ℃</dt><dd>{row.Temperature}</dd>
                <dt>Humidity</dt><dd>{row.Humidity}</dd>
                <dt>Cloudiness</dt><dd>{row.Cloudiness}</dd>
                </dl>
                """
    city_info.append(info_box)

In [10]:
figure_layout = {
    'width': '950px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(map_type="TERRAIN", layout=figure_layout)

coordinates = ideal_cities[["Latitude", "Longitude"]]

marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

### Create New DataFrame

In [11]:
hotel_df = ideal_cities

### Create Function to Obtain Country

- Using the latitude & longitude of the city
- Make a request to Google API and capture the corresponding country

In [12]:
def getCountryForCity(lat, long, df):    
    rtnCountry = ""
    
    params = {"key": g_key}

    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    lat = df["Latitude"][indx]
    lng = df["Longitude"][indx]

    # update address key value
    params['latlng'] = f"{lat},{lng}"

    # make request
    thing = requests.get(base_url, params=params)

    thing = thing.json()
    thing["results"][0]["address_components"]


    for addressComp in thing["results"][0]["address_components"]:
        if addressComp["types"][0] == "country":
            rtnCountry = addressComp["long_name"]
            
    return rtnCountry

Store Countries in a List

In [13]:
countries = []

for indx, row in hotel_df.iterrows():
    countries.append(getCountryForCity(row.Latitude, row.Longitude, hotel_df))

### Create Function to obtain Hotel Details

- store the hotel details (names, addresses & ratings) into lists
- add the lists as columns to the dataframe

In [14]:
def getHoteldetails (df):
    names = []
    addresses = []
    ratings = []
    new_df = pd.DataFrame()
    
    params = {
        "radius": 5000,
        "types": "hotel",
        "keyword": "hotel",
        "key": g_key
    }

    for index, row in df.iterrows():
        lat = row["Latitude"]
        lng = row["Longitude"]

        params["location"] = f"{lat},{lng}"
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        name_address = requests.get(base_url, params=params)
        name_address = name_address.json()

        try:
            names.append(name_address["results"][0]["name"])
            addresses.append(name_address["results"][0]["vicinity"])
            ratings.append(name_address["results"][0]["rating"])
        except (KeyError, IndexError):
            print("Missing field/result... skipping.")
            names.append("")
            addresses.append("")
            ratings.append("")
    
    new_df["Hotels"] = names
    new_df["Addresses"] = addresses
    new_df["Ratings"] = ratings
    
    return new_df

In [15]:
hotel_details = getHoteldetails(hotel_df)

Missing field/result... skipping.
Missing field/result... skipping.


### Update Ideal Cities DataFrame

- add columns with Country, Hotel, Hotel Address and Rating

In [16]:
hotel_df["Country"] = countries
hotel_df["Hotel"] = hotel_details["Hotels"]
hotel_df["Address"] = hotel_details["Addresses"]
hotel_df["Ratings"] = hotel_details["Ratings"]
hotel_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Pressure,Wind Speed,Cloudiness,Country,Hotel,Address,Ratings
0,soto la marina,23.77,-98.22,25.00,80,1014,5.36,0,Mexico,Hotel Los Generales INN,"Pablo L. Sidar SN-S TALLER MECANICO BERNAL, Ju...",4.3
1,vila velha,-20.33,-40.29,21.18,94,1018,1.50,0,Brazil,Golden Tulip Porto Vitória,"Av. Nossa Sra. dos Navegantes, 635 - Enseada d...",4.6
2,redlands,34.06,-117.18,22.24,52,1008,1.79,0,United States,Ayres Hotel Redlands,"1015 W Colton Ave, Redlands",4.5
3,katsuura,35.13,140.30,24.27,60,1013,6.70,0,Japan,Katsuura Hilltop Hotel & Residence,"441-1 Sawakura, Katsuura",3.6
4,tiznit,29.58,-9.50,21.07,44,1017,0.70,0,Morocco,,,


### Map of Hotels

- include pop-up with details on ideal cities

In [17]:
city_info = []

for indx, row in hotel_df.iterrows():
    info_box = f"""
                <dl>
                <dt>Hotel Name</dt><dd>{row.Hotel}</dd>
                <dt>City</dt><dd>{row.City}</dd>
                <dt>Country</dt><dd>{row.Country}</dd>
                <dt>Rating</dt><dd>{row.Ratings}</dd>
                </dl>
                """
    
    city_info.append(info_box)

In [18]:
figure_layout = {
    'width': '950px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

hotel_locale = hotel_df[["Latitude", "Longitude"]]
hotel_temp = hotel_df["Temperature"]

# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN", layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(hotel_locale, weights=hotel_temp, dissipating=True)
fig.add_layer(heat_layer)

coordinates = hotel_df[["Latitude", "Longitude"]]
marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…